In [80]:
from natasha import Segmenter, NewsEmbedding, NewsNERTagger, Doc

segmenter = Segmenter()
emb = NewsEmbedding()
ner = NewsNERTagger(emb)

SPEECH_VERB_ROOTS = [
    'сказ', 'заяв', 'отмет', 'подчеркн', 'сообщ',
    'добав', 'поясн', 'рассказ', 'указ', 'коммент', 'пис'
]

ROLES = {
    'министр финансов',
    'президент', 'министр', 'губернатор',
    'депутат', 'премьер', 'глава', 'директор',
    'артист', 'лауреат',
    'профессор', 'академик', 'редактор', 'журналист',
    'пресс-секретарь',
    'спикер', 'сенатор', 'конгрессмен', 'мэр',
    'генерал', 'полковник', 'капитан',
    'актер', 'актриса', 'режиссер', 'продюсер',
    'писатель', 'поэт', 'художник',
    'ученый', 'исследователь', 'инженер',
}

def extract_filtered_quotes(text, context_chars=50, min_words=3):
    quotes = []
    spans = []
    start = None

    # 1. Извлечение цитат
    for i, ch in enumerate(text):
        if ch == '"':
            if start is None:
                start = i + 1
            else:
                quote = text[start:i].strip()
                quotes.append(quote)
                spans.append((start, i))
                start = None

    if not quotes:
        return []

    # 2. NER
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner)
    persons = [s for s in doc.spans if s.type == 'PER']

    results = []

    for quote, (qs, qe) in zip(quotes, spans):

        is_short = len(quote.split()) < min_words

        # 3. Контекст
        left_idx = max(0, qs - context_chars)
        right_idx = min(len(text), qe + context_chars)
        context = text[left_idx:right_idx].lower()

        # 4. Проверка глагола речи
        if not any(root in context for root in SPEECH_VERB_ROOTS):
            continue

        author = None

        # 5. Поиск PER слева (без глагола речи между)
        per_candidates = []

        for p in persons:
            if p.stop <= qs and qs - p.stop < 150:
                between = text[p.stop:qs].lower()
                # если между PER и цитатой есть глагол речи — PER не автор
                if any(root in between for root in SPEECH_VERB_ROOTS):
                    continue
                per_candidates.append(p)

        if per_candidates:
            p = min(per_candidates, key=lambda p: qs - p.stop)
            author = text[p.start:p.stop]

        # 6. Если PER не найден — ищем роль
        if not author:
            closest_verb_idx = -1

            for root in SPEECH_VERB_ROOTS:
                idx = text[:qs].rfind(root)
                if idx > closest_verb_idx:
                    closest_verb_idx = idx

            if closest_verb_idx != -1:
                sentence_start = max(
                    text[:closest_verb_idx].rfind('.'),
                    text[:closest_verb_idx].rfind('!'),
                    text[:closest_verb_idx].rfind('?'),
                    text[:closest_verb_idx].rfind(':')
                )

                snippet = text[sentence_start + 1:closest_verb_idx].lower()

                matches = [role for role in ROLES if role in snippet]
                if matches:
                    author = max(matches, key=len).capitalize()

        if not author:
            continue

        # 7. Отсев мусорных коротких заголовков
        if quote.istitle() and len(quote.split()) <= 4:
            continue

        if is_short and not author:
            continue

        results.append({
            "quote": quote,
            "author": author
        })

    return results


In [82]:
TEST_TEXTS = [

    # 1. Классика: имя + глагол речи
    'Президент Путин заявил: "Мы продолжим экономические реформы в ближайшие годы".',

    # 2. Роль без имени
    'Министр финансов отметил: "Инфляция замедляется".',

    # 3. Имя + роль + сказал
    'Губернатор Иванов сказал: "Регион справится с вызовами".',

    # 4. Ложная кавычка (не речь)
    'Он посетил ресторан "Белый кролик" и остался доволен.',

    # 5. Имя далеко, глагол ближе
    'Президент Путин посетил форум. Министр заявил: "Решение принято".',

    # 6. Две цитаты — два автора
    (
        'Президент Путин заявил: "Экономика растет". '
        'Министр финансов добавил: "Бюджет исполнен с профицитом".'
    ),

    # 7. Короткая, но валидная
    'Пресс-секретарь сообщил: "Все готово".',

    # 8. Короткая и мусорная
    'Артист сказал: "Невероятно!".',

    # 9. Автор только ролью
    'Депутат подчеркнул: "Закон будет доработан ко второму чтению".',

    # 10. Писатель (глагол "писал")
    'Александр Сергеевич Пушкин писал: "Я вас любил: любовь еще, быть может...".',

    # 11. Автор справа — не должен находиться
    '"Это важный шаг", — заявил министр.',

    # 12. Глагол речи без автора
    'Отмечено: "Результаты будут опубликованы позже".',

    # 13. Два PER слева, правильный — ближний
    'Президент Путин и министр Иванов заявили. Иванов добавил: "Работа продолжается".',

    # 14. Имя есть, но нет глагола речи
    'Президент Путин посетил заседание. "Решение принято".',

    # 15. Несколько предложений, автор не должен протекать
    (
        'Президент Путин выступил на форуме. '
        'Он посетил выставку. '
        'Министр финансов заявил: "Инфляция под контролем".'
    ),

    # 16. Роль составная
    '"Дефицит бюджета снижается" - сообщил министр финансов.',

    # 17. Артист + сказал
    'Народный артист сказал: "Это было незабываемо!".',

    # 18. Перечисление ролей
    'Премьер и министр финансов заявили. Министр финансов отметил: "Решение согласовано".',

    # 19. Глагол речи после роли
    'Губернатор области сообщил: "Ситуация стабильная".',

    # 20. Вообще не речь
    'На табличке было написано: "Вход воспрещен".'
]
for i, text in enumerate(TEST_TEXTS, 1):
    print(f"\n=== TEST {i} ===")
    print(text)
    print(extract_filtered_quotes(text))



=== TEST 1 ===
Президент Путин заявил: "Мы продолжим экономические реформы в ближайшие годы".
[{'quote': 'Мы продолжим экономические реформы в ближайшие годы', 'author': 'Президент'}]

=== TEST 2 ===
Министр финансов отметил: "Инфляция замедляется".
[{'quote': 'Инфляция замедляется', 'author': 'Министр финансов'}]

=== TEST 3 ===
Губернатор Иванов сказал: "Регион справится с вызовами".
[{'quote': 'Регион справится с вызовами', 'author': 'Губернатор'}]

=== TEST 4 ===
Он посетил ресторан "Белый кролик" и остался доволен.
[]

=== TEST 5 ===
Президент Путин посетил форум. Министр заявил: "Решение принято".
[{'quote': 'Решение принято', 'author': 'Министр'}]

=== TEST 6 ===
Президент Путин заявил: "Экономика растет". Министр финансов добавил: "Бюджет исполнен с профицитом".
[{'quote': 'Экономика растет', 'author': 'Президент'}, {'quote': 'Бюджет исполнен с профицитом', 'author': 'Министр финансов'}]

=== TEST 7 ===
Пресс-секретарь сообщил: "Все готово".
[{'quote': 'Все готово', 'author': 